# DX 704 Week 2 Project

This week's project will analyze fresh strawberry price data for a hypothetical "buy low, freeze, and sell high" business.
Strawberries show strong seasonality in their prices compared to other fruits.

![](https://ers.usda.gov/sites/default/files/_laserfiche/Charts/61401/oct14_finding_plattner_fig01.png)

Image source: https://www.ers.usda.gov/amber-waves/2014/october/seasonal-fresh-fruit-price-patterns-differ-across-commodities-the-case-of-strawberries-and-apples

You are considering a business where you buy strawberries when the prices are very low, carefully freeze them, even more carefully defrost them, and then sell them when the prices are high.
You will forecast strawberry price time series and then use them to tactically pick times to buy, freeze, and sell the strawberries.

The full project description, a template notebook, and raw data are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-02


### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Backtest Strawberry Prices

Read the provided "strawberry-prices.tsv" with data from 2000 through 2024.
This data is based on data from the U.S. Bureau of Statistics, but transformed so the ground truth is not online.
https://fred.stlouisfed.org/series/APU0000711415

Use the data for 2000 through 2023 to predict monthly prices in 2024.
Spend some time to make sure you are happy with your methodology and prediction accuracy, since you will reuse the methodology to forecast 2025 next.
Save the 2024 backtest predictions as "strawberry-backtest.tsv" with columns month and price.


In [4]:
# YOUR CHANGES HERE

# Imports
import pandas as pd
import numpy as np
# from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

# Import strawberry-prices.tsv
strawberry_prices = pd.read_csv("strawberry-prices.tsv", sep="\t")
strawberry_prices["month"] = pd.to_datetime(strawberry_prices["month"])

# Limit to 2020 to 2023
mask_train_2020_2023 = (strawberry_prices['month'] >= '2020-01-01') & (strawberry_prices['month'] <= '2023-12-01')
strawberry_prices_2020_2023 = strawberry_prices[mask_train_2020_2023].copy()

# Feature engineering to use month dummies and time
strawberry_prices_2020_2023["month_num"] = strawberry_prices_2020_2023["month"].dt.month
strawberry_prices_2020_2023_X = pd.get_dummies(strawberry_prices_2020_2023["month_num"], prefix="month", drop_first=True)
strawberry_prices_2020_2023_X["time"] = (strawberry_prices_2020_2023["month"].astype("int64") // 10**9)
strawberry_prices_2020_2023_y = strawberry_prices_2020_2023["price"].to_numpy()

# Fit a SVR model
prices_2024_model = make_pipeline(SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1))
prices_2024_model.fit(strawberry_prices_2020_2023_X, strawberry_prices_2020_2023_y)

# Cross-validation
cv_mse = cross_val_score(prices_2024_model, strawberry_prices_2020_2023_X, strawberry_prices_2020_2023_y, scoring='neg_mean_squared_error', cv=5)
print("Mean CV RMSE:", np.sqrt(-cv_mse.mean()))

# Evaluate fit on the training data
y_train_pred = prices_2024_model.predict(strawberry_prices_2020_2023_X)
print("Training MSE:", mean_squared_error(strawberry_prices_2020_2023_y, y_train_pred))
print("Training R^2:", r2_score(strawberry_prices_2020_2023_y, y_train_pred))

# Predict for 2024
months_2024 = pd.date_range(start='2024-01-01', end='2024-12-01', freq='MS')
X_2024 = months_2024.view("int64").reshape(-1, 1)
y_2024_pred = prices_2024_model.predict(X_2024)

backtest_df = pd.DataFrame({
    'month': months_2024,
    'price': y_2024_pred
})

# Evaluate the backtest against actual 2024 values
mask_2024_actual = (strawberry_prices["month"] >= "2024-01-01") & (strawberry_prices["month"] <= "2024-12-01")
actual_2024_prices = strawberry_prices.loc[mask_2024_actual].copy()

mse_2024 = mean_squared_error(actual_2024_prices['price'], backtest_df['price'])
r2_2024 = r2_score(actual_2024_prices['price'], backtest_df['price'])

print("2024 Mean Squared Error:", mse_2024)
print("2024 R^2 Score:", r2_2024)

# Save the backtest results
backtest_df.to_csv("strawberry-backtest.csv", index=False)

Mean CV RMSE: 0.5890748554799441
Training MSE: 0.008942821756249967
Training R^2: 0.9693956570814817


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(


ValueError: X has 1 features, but SVR is expecting 12 features as input.

Submit "strawberry-backtest.tsv" in Gradescope.

## Part 2: Backtest Errors

What are the mean and standard deviation of the residuals between your backtest predictions and the ground truth? (If your mean is not close to zero, then you may be missing a long term trend.)

Write the mean and standard deviation to a file "backtest-accuracy.tsv" with two columns, mean and std.

In [ ]:
# YOUR CHANGES HERE

...

Submit "backtest-accuracy.tsv" in Gradescope.

## Part 3: Forecast Strawberry Prices

Use all the data from 2000 through 2024 to predict monthly prices in 2025 using the same methodology from part 1.
Make a monthly forecast for each month of 2025 and save it as "strawberry-forecast.tsv" with columns for month and price.

In [ ]:
# YOUR CHANGES HERE

...

Submit "strawberry-forecast.tsv" in Gradescope.

## Part 4: Buy Low, Freeze and Sell High

Using your 2025 forecast, analyze the profit picking different pairs of months to buy and sell strawberries.
Maximize your profit assuming that it costs &dollar;0.20 per pint to freeze the strawberries, &dollar;0.10 per pint per month to store the frozen strawberries and there is a 10% price discount from selling previously frozen strawberries.
So, if you buy a pint of strawberies for &dollar;1, freeze them, and sell them for &dollar;2 three months after buying them, then the profit is &dollar;2 * 0.9 - &dollar;1 - &dollar;0.20 - &dollar;0.10 * 3 = &dollar;0.30 per pint.
To evaluate a given pair of months, assume that you can invest &dollar;1,000,000 to cover all costs, and that you buy as many pints of strawberries as possible.

Write the results of your analysis to a file "timings.tsv" with columns for the buy_month, sell_month, pints_purchased, and expected_profit.

In [ ]:
# YOUR CHANGES HERE

...

Submit "timings.tsv" in Gradescope.

## Part 5: Strategy Check

What is the best profit scenario according to your previous timing analysis?
How much does that profit change if the sell price is off by one standard deviation from your backtest analysis?
(Variation in the sell price is more dangerous because you can see the buy price before fully committing.)

Write the results to a file "check.tsv" with columns best_profit and one_std_profit.

In [ ]:
# YOUR CHANGES HERE

...

Submit "check.tsv" in Gradescope.

## Part 6: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgements.txt" in Gradescope.

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.